In [1]:
import numpy as np

In [2]:
def dft(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)

In [3]:
x = np.random.random(1024)
np.allclose(dft(x), np.fft.fft(x))

True

In [4]:
%timeit dft(x)
%timeit np.fft.fft(x)

72.4 ms ± 422 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
17.1 µs ± 514 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
def fft(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    if N % 2 > 0:
        raise ValueError("must be a power of 2")
    elif N <= 2:
        return dft(x)
    else:
        X_even = fft(x[::2])
        X_odd = fft(x[1::2])
        terms = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate([X_even + terms[:int(N/2)] * X_odd,
                               X_even + terms[int(N/2):] * X_odd])

In [6]:
x = np.random.random(1024)
np.allclose(fft(x), np.fft.fft(x))

True

In [7]:
%timeit dft(x)
%timeit fft(x)
%timeit np.fft.fft(x)

73.9 ms ± 2.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
12.1 ms ± 76.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
17 µs ± 61.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [8]:
def fft_v(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    if np.log2(N) % 1 > 0:
        raise ValueError("must be a power of 2")
        
    N_min = min(N, 2)
    
    n = np.arange(N_min)
    k = n[:, None]
    M = np.exp(-2j * np.pi * n * k / N_min)
    X = np.dot(M, x.reshape((N_min, -1)))
    while X.shape[0] < N:
        X_even = X[:, :int(X.shape[1] / 2)]
        X_odd = X[:, int(X.shape[1] / 2):]
        terms = np.exp(-1j * np.pi * np.arange(X.shape[0])
                        / X.shape[0])[:, None]
        X = np.vstack([X_even + terms * X_odd,
                    X_even - terms * X_odd])
    return X.ravel()

In [9]:
x = np.random.random(1024)
np.allclose(fft_v(x), np.fft.fft(x))

True

In [10]:
%timeit fft(x)
%timeit fft_v(x)
%timeit np.fft.fft(x)

11.8 ms ± 94 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
302 µs ± 4.29 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
16.7 µs ± 56.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
